In [1]:
import re
import os
import csv
import nltk
import pickle
import numpy as np
import heapq
from scipy.spatial import distance
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from collections import Counter
from heapq import heappush
N_doc=26543

In [2]:
tokenizer = RegexpTokenizer(r'[a-z]+') #Change this line by removing 0-9 if we don't want numbers in the plot tokens.
stop_words = set(stopwords.words("english"))
stemmer= PorterStemmer()

To see the code used to create these pkl files look at the "DataStructures" notebook

In [3]:
with open('inverted_index_1.pkl', 'rb') as handle:
    inverted_index = pickle.load(handle)
with open('vocabulary.pkl', 'rb') as handle:
    vocabulary = pickle.load(handle)
with open('vocabulary2.pkl', 'rb') as handle:
    vocabulary2 = pickle.load(handle)
with open('tfIdf_index.pkl', 'rb') as handle:
    tfIdf_index = pickle.load(handle)
with open('BookTokens.pkl', 'rb') as handle:
    BookTokens = pickle.load(handle)

## 2.1 Conjunctive query

In [5]:
query = input()
query = tokenizer.tokenize(query.lower())
query_stems = [stemmer.stem(word) for word in query if word not in stop_words]

query_stem_test=query_stems
query_stems=[]

#Checking if input stems exists in the vocabulary

for word in query_stem_test:
    try:
        vocabulary[word]
        query_stems.append(word)
    except KeyError:
        print("Stem",word,"not found. It will be ignored.")


        
        
temp=set()
if len(query_stems)>0:
    temp=inverted_index[vocabulary[query_stems[0]]]
    for stem in query_stems:
        temp=temp.intersection(inverted_index[vocabulary[stem]])

matching_books=list(sorted(temp))

for i in matching_books:
    with open('articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
            temp = csv.DictReader(file, delimiter = '\t')
            for row in temp:
                print("BookTitle:",row["bookTitle"])
                print("Plot:")
                print(row["Plot"])
                print("Url:",row["Url"])
                print()

Defense Against
BookTitle: Harry Potter and the Order of the Phoenix
Plot:
There is a door at the end of a silent corridor. And it’s haunting Harry Pottter’s dreams. Why else would he be waking in the middle of the night, screaming in terror? Harry has a lot on his mind for this, his fifth year at Hogwarts: a Defense Against the Dark Arts teacher with a personality like poisoned honey; a big surprise on the Gryffindor Quidditch team; and the looming terror of the Ordinary Wizarding Level exams. But all these things pale next to the growing threat of He-Who-Must-Not-Be-Named - a threat that neither the magical government nor the authorities at Hogwarts can stop. As the grasp of darkness tightens, Harry must discover the true depth and strength of his friends, the importance of boundless loyalty, and the shocking price of unbearable sacrifice. His fate depends on them all.
Url: https://www.goodreads.com/book/show/2.Harry_Potter_and_the_Order_of_the_Phoenix

BookTitle: The Stand
Plot:
Thi

Url: https://www.goodreads.com/book/show/35642518-for-the-love-of-armin

BookTitle: Firelight
Plot:
A hidden truth. Mortal enemies. Doomed love. Marked as special at an early age, Jacinda knows her every move is watched. But she longs for freedom to make her own choices. When she breaks the most sacred tenet among her kind, she nearly pays with her life. Until a beautiful stranger saves her. A stranger who was sent to hunt those like her. For Jacinda is a draki, a descendant of dragons whose greatest defense is her secret ability to shift into human form. Forced to flee into the mortal world with her family, Jacinda struggles to adapt to her new surroundings. The only bright light is Will. Gorgeous, elusive Will who stirs her inner draki to life. Although she is irresistibly drawn to him, Jacinda knows Will's dark secret: He and his family are hunters. She should avoid him at all costs. But her inner draki is slowly slipping away;if it dies she will be left as a human forever. She'll d

BookTitle: The Rithmatist
Plot:
The Rithmatist, Brandon Sanderson's New York Times bestselling epic teen adventure is now available in paperback. More than anything, Joel wants to be a Rithmatist. Rithmatists have the power to infuse life into two-dimensional figures known as Chalklings. Rithmatists are humanity’s only defense against the Wild Chalklings. Having nearly overrun the territory of Nebrask, the Wild Chalklings now threaten all of the American Isles. As the son of a lowly chalkmaker at Armedius Academy, Joel can only watch as Rithmatist students learn the magical art that he would do anything to practice. Then students start disappearing—kidnapped from their rooms at night, leaving trails of blood. Assigned to help the professor who is investigating the crimes, Joel and his friend Melody find themselves on the trail of an unexpected discovery—one that will change Rithmatics—and their world—forever. A New York Times Book Review Notable Children's Book of 2013
Url: https://www

BookTitle: The Witness
Plot:
Daughter of a controlling mother, Elizabeth finally let loose one night, drinking at a nightclub and allowing a strange man's seductive Russian accent lure her to a house on Lake Shore Drive. The events that followed changed her life forever. Twelve years later, the woman known as Abigail Lowery lives on the outskirts of a small town in the Ozarks. A freelance programmer, she designs sophisticated security systems -- and supplements her own security with a fierce dog and an assortment of firearms. She keeps to herself, saying little, revealing nothing. But Abigail's reserve only intrigues police chief Brooks Gleason. Her logical mind, her secretive nature, and her unromantic viewpoints leave him fascinated but frustrated. He suspects that Abigail needs protection from something -- and that her elaborate defenses hide a story that must be revealed. With a quirky, unforgettable heroine and a pulse-pounding plotline, Nora Roberts presents a riveting new read t

BookTitle: Spark of Life: A Novel of Resistance
Plot:
SPARK OF LIFE 509 is a political prisoner in a German concentration camp. For ten years, he has persevered in the most hellish conditions. Deathly weak, he still has his wits about him and he senses that the end of the war is near. If he and the other living corpses in his barracks can hold on for liberation--or force their own--then their suffering will not have been in vain. Now the SS who run the camp are ratcheting up the terror. But their expectations are jaded and their defenses are down. It is possible that the courageous, yet terribly weak prisoners have just enough left in them to resist. And if they die fighting, they will die on their own terms, cheating the Nazis out of their devil's contract.
Url: https://www.goodreads.com/book/show/64097.Spark_of_Life

BookTitle: What Was She Thinking? [Notes on a Scandal]
Plot:
Schoolteacher Barbara Covett has led a solitary life until Sheba Hart, the new art teacher at St. George's, 

...

BookTitle: Our Virtuous Republic: The Forgotten Clause in the American Social Contract
Plot:
Is it no longer a "self-evident" truth that government's nature is to become tyrannical? "Our Virtuous Republic" is the book the establishment does not want Americans on the right, left, or center to read. The argument is a vociferous and comprehensive defense of American traditionalism, a lesson in American citizenship, and a call for a return to civil society. Conservatives academics and politicians have failed to make a decisive argument for our founding principles. As a result, the progressive narrative has attached an unsubstantiated backward stigma to the ideals born of the American mainstream Protestant ethic, such as civic duty, obligation and other elements of American traditionalism that are necessary for a healthy society. Baris uses psychology, logic, economics, history, and science to deconstruct the progressive argument to its regressive core. Breaking through the superficial, he 

BookTitle: The Ghost Brigades
Plot:
The Ghost Brigades are the Special Forces of the Colonial Defense Forces, elite troops created from the DNA of the dead and turned into the perfect soldiers for the CDF's toughest operations. They’re young, they’re fast and strong, and they’re totally without normal human qualms. The universe is a dangerous place for humanity—and it's about to become far more dangerous. Three races that humans have clashed with before have allied to halt our expansion into space. Their linchpin: the turncoat military scientist Charles Boutin, who knows the CDF’s biggest military secrets. To prevail, the CDF must find out why Boutin did what he did. Jared Dirac is the only human who can provide answers -- a superhuman hybrid, created from Boutin's DNA, Jared’s brain should be able to access Boutin's electronic memories. But when the memory transplant appears to fail, Jared is given to the Ghost Brigades. At first, Jared is a perfect soldier, but as Boutin’s memories s

BookTitle: The Other Side of Darkness
Plot:
Once again, I have kept the demons at bay. As a wife and mother, Ruth knows her prayers are crucial to her family’s spiritual welfare. She stands between her precious children and the evil one, doing battle in prayer. She can’t afford to be careless. Thankfully, she has powerful allies: Pastor Glenn, New Life Christian School where her daughters Mary and Sarah attend, and the inner circle at Arbor Drive Fellowship. They all reinforce her careful nurturance of her children. If only her husband, Rick, understood that. He’s exasperated about the money Ruth keeps spending for the church and school. Doesn’t he see that these are their best defenses in shielding their children from the dangers of the world? But the forces that threaten Ruth’s faith, her family–her very life–are not the ones she expects. Ruth doesn’t realize that her heartfelt desire to obey God is mingled with dangerous currents of OCD–Obsessive Compulsive Disorder. Her own strateg

BookTitle: The Death of Captain Marvel
Plot:
Captain Marvel wielded his cosmic powers in defense of the galaxy, but exposure to a carcinogenic nerve gas causes him to succumb to an incurable cancer.
Url: https://www.goodreads.com/book/show/1014568.The_Death_of_Captain_Marvel

BookTitle: Wild Chase
Plot:
Music is a Moral Law, It Gives Soul to the Universe- Plato Sometimes it doesn't matter how long you know a person. The effect they can have on you can happen after four seconds or four years. In Chase Taylor's case - it's both. Avery McGuire doesn't just flip on a light inside of her - he sets a blaze to everything holding her back. After only a week together, he has to leave, but he strikes a match before walking away that won't easily burn out. Four years later, Chase finds herself face to face with the boy -now rock star - who changed everything. The same one she still can't admit to falling head over heels in love with. She has no idea that her world is about to get rocked, in more 

Plot:
Christmas Eve finds renowned, but now disgraced, defense lawyer George Able paying a final visit to his two children, Brian and Sandy. It is a final visit of a different sort to his estranged wife and her lover: George has a .38 in his coat pocket. Later that night, he finds himself on a deserted highway in deep dark country with missing bullets and missing time. It looks like a Christmas Miracle, especially when he meets a beautiful redhead who offers him a second chance. But the seemingly naive Red Morgan proves to be a troubling mix of love, lethality and, possibly, lunacy. When George’s daughter is kidnapped by a man who might be associated with Morgan, the Able family and the criminal justice system are faced with a threat they are not equipped to handle. As a rescue attempt snowballs into a crime spree from the top of Illinois to its tip, a family must put aside its problems even as moral compromises undermine their good intentions. And while Red Morgan is too useful to sen

## 2.2 Conjunctive query and ranking score

In [25]:
query = input()
query = tokenizer.tokenize(query.lower())
query_stems = [stemmer.stem(word) for word in query if word not in stop_words]

query_stem_test=query_stems
query_stems=[]

#Checking if input stems exists in the vocabulary

for word in query_stem_test:
    try:
        vocabulary[word]
        query_stems.append(word)
    except KeyError:
        print("Stem",word,"not found. It will be ignored.")

query_stems=list(dict.fromkeys([x for x in query_stems])) #Removing possible similarities

##########################
temp=set()

if len(query_stems)>0:
    temp=inverted_index[vocabulary[query_stems[0]]]
    for stem in query_stems:
        temp=temp.intersection(inverted_index[vocabulary[stem]])

matching_books=list(sorted(temp))

#Calculating tfIdf for the query.
query_tfIdf=[]

for word in query_stems:
    query_tfIdf.append((vocabulary[word],np.log(N_doc/vocabulary2[vocabulary[word]])))
query_tfIdf.sort()

query_tfIdf=dict((x,y) for x,y in query_tfIdf)



BooksWithScore=[] #Initializing the heap structure

for book in matching_books:
    doc_vector=[]
    query_vector=[]
    for word_id in BookTokens[book]:
        doc_vector.append(word_id[1])
        if word_id[0] in query_tfIdf:
            query_vector.append(1)
        else:
            query_vector.append(0)

    doc_vector=np.array(doc_vector)
    query_vector=np.array(query_vector)
    cos_similarity=1-distance.cosine(doc_vector,query_vector)
    
    heappush(BooksWithScore, (book,cos_similarity))


#print(BooksWithScore)

top_k_books=heapq.nlargest(5,BooksWithScore,key=lambda x:x[1])

for book in top_k_books:
    i=book[0]
    with open('articles/article_' + str(i) +'.tsv', 'r', encoding="utf-8") as file:
            temp = csv.DictReader(file, delimiter = '\t')
            for row in temp:
                print("BookTitle:",row["bookTitle"])
                print("Plot:")
                print(row["Plot"])
                print("Url:",row["Url"])
                print("Score:",book[1])
                print()
#heapq.heapify(H)

survival games
BookTitle: The Warden
Plot:
Alice has led a normal life up until now. She wakes up finding herself trapped in a sick game of survival within the walls of an old asylum. She has to fight her way out while facing psychotic enemies, no one said they were all living psychopaths though... When she discovers how she got there she is forced to make difficult decisions. Will Alice survive the horror games?
Url: https://www.goodreads.com/book/show/33818812-the-warden
Score: 0.3592488971900667

BookTitle: Devil's Own
Plot:
After surviving slavery, Aiden MacAlpin has nothing but thoughts of vengeance. When his tutor Elspeth learns a secret to his past, it thrusts them both into a game of passion and deception that neither may survive.
Url: https://www.goodreads.com/book/show/8705483-devil-s-own
Score: 0.27980861944504043

BookTitle: The Quillan Games
Plot:
LET THE GAMES BEGIN.... Quillan is a territory on the verge of destruction. The people have lost control of their own future an